# Classifying long vs short life in LFP/Graphite cells 👬

### In this notebook, we are using binary logistic regression to classify long vs short lived cells, based on the efforts of Severson et. al in "Data-driven prediction of battery cycle life before capacity degradation" 💻
---------------------------------

## We must again first load the data 📊

In [1]:
import pickle

def load_batches(paths):
    all_cells = {}
    for path in paths:
        with open(path, 'rb') as f:
            batch = pickle.load(f)
        all_cells.update(batch)
    return all_cells

batch_paths = ['../data/batch1.pkl', '../data/batch2.pkl', '../data/batch3.pkl']
cells = load_batches(batch_paths)

## Now, we need to make and train our models 👨‍🍼

To get started, we'll load functions from our other notebook.

In [2]:
def load_functions_from_ipynb(path, function_names, namespace=None):
    import nbformat, ast

    if namespace is None:
        namespace = {}

    with open(path) as f:
        nb = nbformat.read(f, as_version=4)

    for cell in nb.cells:
        if cell.cell_type != "code":
            continue

        try:
            parsed = ast.parse(cell.source)
            for node in parsed.body:
                if isinstance(node, ast.FunctionDef) and node.name in function_names:
                    print(f"Loading function: {node.name}")  # 👈 helpful debug
                    exec(compile(ast.Module([node], []), filename="<ast>", mode="exec"), namespace)
        except Exception as e:
            print(f"Skipping cell due to error: {e}")  # 👈 helpful debug

    

    return namespace


function_names = ['extract_features_simple',
                  'extract_features_full',
                  'create_X_y_arrays',
                  'split_and_fit', ]

ns = load_functions_from_ipynb(r"1_Cycle-life-prediction-using-interpretable-ML.ipynb", function_names)

Loading function: extract_features_simple
Loading function: create_X_y_arrays
Loading function: split_and_fit
Loading function: extract_features_full


Now, we need to create the feature and target arrays.

In [5]:
import numpy as np
ns['np'] = np
ns['cells'] = cells
X_simple, y_numeric = ns['create_X_y_arrays'](ns['extract_features_simple'])
X_full = ns['create_X_y_arrays'](ns['extract_features_full'])[0]

cycle_threshold = 550 # number of cycles used to say if it's a short or long-lived cell
y = 